In [ ]:
import pandas as pd
import numpy as np
import re   #for additional splitting methods. Used to remove the numbering on cities names
from simpledbf import Dbf5    #used to extract cities' positions from a dbf table at https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27385-localidades.html?=&t=acesso-ao-produto
import os
import glob

import matplotlib.pyplot as plt
import itertools

# --- Dengue, Zika, Influenza ---

In [ ]:
#Configs
disease = 'dengue'
path = 'your_path/' + disease + '/'
year = 2018

## Data for each city

###If using dengue of zika data:

In [ ]:
lines_to_skip = [0,1,2] + list(range(3168, 5000, 1))  #change according to the file provided in "[Dengue/Zika] Line skips" below

In [ ]:
df = pd.read_csv(str(path + 'filename' + '.csv'), sep=';', encoding='latin1', skiprows=lines_to_skip)

# === Uncomment the lines below as needed ===
# --- Dengue ---
#df.drop(columns=['Em Branco/ign', 'Total', 'Semana 53'], inplace=True)   #2014
#df.drop(columns=['Em Branco/ign', 'Total'], inplace=True)   #2015, 2020, 2021
df.drop(columns=['Total'], inplace=True)   #2019, 2017, 2016, 2018, 2019, 2022

# --- Zika ---
#df.drop(columns=['Em Branco/ign', 'Total', 'Semana 53'], inplace=True)   #2014
#df.drop(columns=['Em Branco/ign', 'Total'], inplace=True)   #2015, 2020, 2021
#df.drop(columns=['Total'], inplace=True)   #2016, 2017, 2018, 2019, 2022

In [ ]:
#Reworking city names to move the unique codes to another column.

renamed = []
city_codes = []
names = list(df['Município de notificação'])

for city in range(0, len(names), 1):
  swp = re.split('(\d+)', names[city])  #split city name in '', number and name
  city_codes.append( int(swp[1].strip()) ) #save each city code for later use.
  renamed.append(swp[2].strip()) #remove whitespace before the name of a city and save result

#Drop the original column and replaces it with the above list
n = df.columns[0]   #index of the city list
df.drop(n, axis = 1, inplace = True)
df[n] = renamed
df['cod_city'] = city_codes

In [ ]:
#Reorganize the placement of the new name list and show the resulting dataframe
#-> The code is duplicated to move both the city names and codes to the beginning of the dataframe.
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df

#replace values listed as "-" with NaN, excluding the cities column (some have "-" in their names)
for col in df.columns:
  if col != 'Município de notificação':
    df[col].replace({'-': np.NaN}, regex=True, inplace=True)

### If using influenza data:

Original download from https://opendatasus.saude.gov.br/dataset/srag-2013-2018

In [ ]:
df = pd.read_csv(str(path + disease + str(year) + '.csv'), delimiter=';', encoding='latin')

In [ ]:
df.DT_NOTIFIC = pd.to_datetime(df.DT_NOTIFIC, format="%d/%m/%Y")
df['Year'] = df['DT_NOTIFIC'].apply(lambda time: time.year)
df['Week'] = df['DT_NOTIFIC'].apply(lambda time: time.week)

df.sort_values(by=['Week', 'ID_MUNICIP'], inplace=True)
#somehow there is more than one year in this dataframe
df.drop(columns='NU_ANO', inplace=True)
df = df.loc[df['Year'] == int(year)]

df.CS_SEXO[df.CS_SEXO == 'M'] = 1
df.CS_SEXO[df.CS_SEXO == 'F'] = 1
df.CS_SEXO[df.CS_SEXO == 'I'] = 1
df['CONTAGEM'] = df.groupby(by=['Week', 'ID_MUNICIP'])['CS_SEXO'].transform('sum')

df.sort_values(by=['DT_NOTIFIC'], inplace=True)
df = df[['DT_NOTIFIC', 'ID_MUNICIP', 'Year', 'Week', 'CONTAGEM']]

## Adding geographic distances

**Important**: Sadly you can't mix the city codes from the IBGE dataset and those from SINAN's. The former have numberings with seven digits (while the latter has six) that do not overlap with latter.

Proof:



```
test = []

for number in list(df_loc['MUN_ORIGEM'].unique()):
  operationTwo = number
  test.append( operationTwo // 10 )

len(set(test) & set(df['cod_city'].unique())) #3710 vs 3774
```



In [ ]:
path = 'your_path/'
dbf = Dbf5(path + 'BR_Localidades_2010_v1.dbf', codec='latin')
df_loc = dbf.to_dataframe()

In [ ]:
#Dictionaty to convert full lenght UF names to codes
#-> for some reason, the original dataframe, df, does not have data for Espirito Santo
uf_code = ['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN',
       'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR', 'SC', 'RS',
       'MS', 'MT', 'GO', 'DF']
uf_names = list(df_loc['NM_UF'].unique())

for i in range(len(uf_code)):
    df_loc['NM_UF'] = df_loc['NM_UF'].replace(uf_names[i],uf_code[i])

In [ ]:
#-> Removing accents
cols = df_loc.select_dtypes(include=[object]).columns
df_loc[cols] = df_loc[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
df_loc = df_loc.loc[df_loc['NM_CATEGOR'] == 'CIDADE']
df_loc['ID_MUNICIP'] = pd.to_numeric(df_loc['CD_GEOCODM'].str[:-1])

df_loc.sort_values(by=['ID_MUNICIP'], inplace=True)

### Run for Influenza

In [ ]:
x_coord = []
y_coord = []
city_to_remove = []

for city in list(df['ID_MUNICIP'].unique()):
  df_city = df_loc.loc[df_loc['ID_MUNICIP'] == city]
  size_city_data = len(df.loc[df['ID_MUNICIP'] == city])
  if (len(df_city) != 0):
    x = df_city['LAT'].values[0]
    y = df_city['LONG'].values[0]
    x_coord.append( np.repeat(x, size_city_data) )
    y_coord.append( np.repeat(y, size_city_data) )
  else:
    city_to_remove.append(city)

x_coord = list(itertools.chain(*x_coord))
y_coord = list(itertools.chain(*y_coord))

for city in city_to_remove:
  df = df[df['ID_MUNICIP'] != city]


#Add the positions found
df['x_coord'] = x_coord
df['y_coord'] = y_coord

df = df[['ID_MUNICIP', 'Week', 'x_coord', 'y_coord', 'CONTAGEM']]

In [ ]:
#expand the dataframe. Is there a better way to do that?
weeks_full_year = list(range(1,53))

swp_id = []
swp_xcoord = []
swp_ycoord = []
swp_contagem = []

for city in df['ID_MUNICIP'].unique():
  cont = 0
  list_weeks = df.loc[df['ID_MUNICIP'] == city]['Week'].values
  weeks_cases = df.loc[df['ID_MUNICIP'] == city]['CONTAGEM'].values

  swp_id.append( np.repeat(city, len(weeks_full_year)) )
  swp_xcoord.append( np.repeat(df.loc[df['ID_MUNICIP'] == city]['x_coord'].values[0], len(weeks_full_year)) )
  swp_ycoord.append( np.repeat(df.loc[df['ID_MUNICIP'] == city]['y_coord'].values[0], len(weeks_full_year)) )

  for value in weeks_full_year:
    if value in list_weeks:
      swp_contagem.append( weeks_cases[cont] )
      cont = cont+1
    else:
      swp_contagem.append(0)

swp_year = np.repeat(int('20' + str(year)), len(swp_contagem))
swp_id = list(itertools.chain(*swp_id))
swp_xcoord = list(itertools.chain(*swp_xcoord))
swp_ycoord = list(itertools.chain(*swp_ycoord))

In [ ]:
data = {'Year': swp_year, 'cod_city':swp_id, 'week': np.tile(weeks_full_year, len(df['ID_MUNICIP'].unique())),
        'x_coord': swp_xcoord, 'y_coord': swp_ycoord, 'cases':swp_contagem}

df_final = pd.DataFrame(data)

### Run for Dengue/Zika

In [ ]:
x_coord = []
y_coord = []
city_to_remove = []

for city in list(df['cod_city'].unique()):
  df_city = df_loc.loc[df_loc['ID_MUNICIP'] == city]
  size_city_data = len(df.loc[df['cod_city'] == city])
  if (len(df_city) != 0):
    x = df_city['LAT'].values[0]
    y = df_city['LONG'].values[0]
    x_coord.append( np.repeat(x, size_city_data) )
    y_coord.append( np.repeat(y, size_city_data) )
  else:
    city_to_remove.append(city)

x_coord = list(itertools.chain(*x_coord))
y_coord = list(itertools.chain(*y_coord))

In [ ]:
for city in city_to_remove:
  df = df[df['cod_city'] != city]

In [ ]:
#rename weeks to a easier to access format.
#-> The other variables (and those from the INMET dataframes) will be translated in another notebook.
cols = list(df.select_dtypes(include=[object]).columns)[1:]

for week in range(0, len(cols), 1):
  df.rename(columns={cols[week]: str('w_'+str(week+1))}, inplace=True)

In [ ]:
#Add the positions found in the previous cell
df['x_coord'] = x_coord
df['y_coord'] = y_coord

df = df[['cod_city', 'x_coord', 'y_coord', 'w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6',
       'w_7', 'w_8', 'w_9', 'w_10', 'w_11', 'w_12', 'w_13', 'w_14', 'w_15',
       'w_16', 'w_17', 'w_18', 'w_19', 'w_20', 'w_21', 'w_22', 'w_23', 'w_24',
       'w_25', 'w_26', 'w_27', 'w_28', 'w_29', 'w_30', 'w_31', 'w_32', 'w_33',
       'w_34', 'w_35', 'w_36', 'w_37', 'w_38', 'w_39', 'w_40', 'w_41', 'w_42',
       'w_43', 'w_44', 'w_45', 'w_46', 'w_47', 'w_48', 'w_49', 'w_50', 'w_51',
       'w_52']]

## Saving results

In [ ]:
#Done! Let's save the resulting data.
df.to_csv(str(path + '/SINAN_' + disease + 'weekly_' + str(year) + '.csv'))

##[Dengue/Zika] Line skips

### Dengue

In [ ]:
#2007:
lines_to_skip = [0,1,2] + list(range(3559, 5000, 1))

#2008:
lines_to_skip = [0,1,2] + list(range(3363, 5000, 1))

#2009:
lines_to_skip = [0,1,2] + list(range(3041, 5000, 1))

#2010:
lines_to_skip = [0,1,2] + list(range(3939, 5000, 1))

#2011:
lines_to_skip = [0,1,2] + list(range(3841, 5000, 1))

#2012:
lines_to_skip = [0,1,2] + list(range(3567, 5000, 1))

#2013:
lines_to_skip = [0,1,2] + list(range(4221, 5000, 1))

#2014:
lines_to_skip = [0,1,2] + list(range(3694, 5000, 1))

#2015:
lines_to_skip = [0,1,2] + list(range(4365, 5000, 1))

#2016:
lines_to_skip = [0,1,2] + list(range(4550, 5000, 1))

#2017:
lines_to_skip = [0,1,2] + list(range(3268, 5000, 1))

#2018:
lines_to_skip = [0,1,2] + list(range(3168, 5000, 1))

#2019:
lines_to_skip = [0,1,2] + list(range(4339, 5000, 1))

#2020:
lines_to_skip = [0,1,2] + list(range(4080, 5000, 1))

#2021:
lines_to_skip = [0,1,2] + list(range(3778, 5000, 1))

### Zika

In [ ]:
#2016:
lines_to_skip = [0,1,2] + list(range(2375, 5000, 1))

#2017:
lines_to_skip = [0,1,2] + list(range(1258, 5000, 1))

#2018:
lines_to_skip = [0,1,2] + list(range(1143, 5000, 1))

#2019:
lines_to_skip = [0,1,2] + list(range(1463, 5000, 1))

#2020:
lines_to_skip = [0,1,2] + list(range(1094, 5000, 1))

#2021:
lines_to_skip = [0,1,2] + list(range(941, 5000, 1))

#2022:
lines_to_skip = [0,1,2] + list(range(1159, 5000, 1))

#2023:
lines_to_skip = [0,1,2] + list(range(1057, 5000, 1))

# --- COVID19 ----

In [37]:
path = "your_path/"
disease = 'covid'
file_list = [os.path.normpath(i) for i in glob.glob(path + "/*.csv")]
main_dataframe = pd.read_csv(file_list[0], delimiter=';')
for i in range(1,len(file_list)):
    df = pd.read_csv(file_list[i], delimiter=';')
    main_dataframe = pd.concat([main_dataframe,df],axis=0)

In [4]:
main_dataframe.columns

Index(['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude',
       'nomeRegiaoSaude', 'data', 'semanaEpi', 'populacaoTCU2019',
       'casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos',
       'Recuperadosnovos', 'emAcompanhamentoNovos', 'interior/metropolitana'],
      dtype='object')

In [38]:
main_dataframe = main_dataframe[['estado', 'municipio', 'codmun', 'data',
                                 'semanaEpi','casosAcumulado', 'casosNovos']]
main_dataframe.dropna(subset=['estado','municipio', 'codmun'], inplace=True)

main_dataframe.set_index('data', inplace=True)
main_dataframe.index = pd.to_datetime(main_dataframe.index)

In [39]:
data_to_concatenate = []
for column in main_dataframe.columns[4:]:
  grouped = main_dataframe.groupby('codmun').resample('1W')[column].sum()
  data_to_concatenate.append(grouped.to_frame())

res = pd.concat(data_to_concatenate, axis=1)

res = res.reset_index(level=[0,1])
res['Year'] = pd.DatetimeIndex(res['data']).year
res['week'] = res['data'].dt.strftime("%U")
res['week'] = pd.to_numeric(res['week'])

##Adding geographic distances

**Important**: Sadly you can't mix the city codes from the IBGE dataset and those from SINAN's. The former have numberings with seven digits (while the latter has six) that do not overlap with latter.

Proof:



```
test = []

for number in list(df_loc['MUN_ORIGEM'].unique()):
  operationTwo = number
  test.append( operationTwo // 10 )

len(set(test) & set(df['cod_city'].unique())) #3710 vs 3774
```



In [21]:
path = 'your_path/'
dbf = Dbf5(path + 'BR_Localidades_2010_v1.dbf', codec='latin')
df_loc = dbf.to_dataframe()

In [22]:
#Dictionaty to convert full lenght UF names to codes
#-> for some reason, the original dataframe, df, does not have data for Espirito Santo
uf_code = ['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN',
       'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR', 'SC', 'RS',
       'MS', 'MT', 'GO', 'DF']
uf_names = list(df_loc['NM_UF'].unique())

for i in range(len(uf_code)):
    df_loc['NM_UF'] = df_loc['NM_UF'].replace(uf_names[i],uf_code[i])

In [27]:
#-> Removing accents
cols = df_loc.select_dtypes(include=[object]).columns
df_loc[cols] = df_loc[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [28]:
df_loc = df_loc.loc[df_loc['NM_CATEGOR'] == 'CIDADE']
df_loc['ID_MUNICIP'] = pd.to_numeric(df_loc['CD_GEOCODM'].str[:-1])

df_loc.sort_values(by=['ID_MUNICIP'], inplace=True)

In [40]:
x_coord = []
y_coord = []
city_to_remove = []

for city in list(res['codmun'].unique()):
  df_city = df_loc.loc[df_loc['ID_MUNICIP'] == city]
  size_city_data = len(res.loc[res['codmun'] == city])
  if (len(df_city) != 0):
    x = df_city['LAT'].values[0]
    y = df_city['LONG'].values[0]
    x_coord.append( np.repeat(x, size_city_data) )
    y_coord.append( np.repeat(y, size_city_data) )
  else:
    city_to_remove.append(city)

x_coord = list(itertools.chain(*x_coord))
y_coord = list(itertools.chain(*y_coord))

In [41]:
for city in city_to_remove:
  res = res[res['codmun'] != city]

In [42]:
#Add the positions found in the previous cell
res['x_coord'] = x_coord
res['y_coord'] = y_coord

,codmun,data,casosAcumulado,casosNovos,Year,week,x_coord,y_coord
0,110001.0,2020-03-29,0.0,0,2020,13,-11.935540,-61.999824
1,110001.0,2020-04-05,0.0,0,2020,14,-11.935540,-61.999824
2,110001.0,2020-04-12,0.0,0,2020,15,-11.935540,-61.999824
3,110001.0,2020-04-19,0.0,0,2020,16,-11.935540,-61.999824
4,110001.0,2020-04-26,0.0,0,2020,17,-11.935540,-61.999824
...,...,...,...,...,...,...,...,...
1097285,530010.0,2023-12-03,6482474.0,258,2023,49,-15.794087,-47.887905
1097286,530010.0,2023-12-10,6484370.0,303,2023,50,-15.794087,-47.887905
1097287,530010.0,2023-12-17,6486391.0,253,2023,51,-15.794087,-47.887905
1097288,530010.0,2023-12-24,6487956.0,150,2023,52,-15.794087,-47.887905


In [ ]:
res.rename(columns={'codmun': 'cod_city', 'casosNovos': 'cases'}, inplace=True)
res = res[['cod_city', 'Year', 'week', 'data', 'cases', 'x_coord', 'y_coord']]
res['cases'].mask(res['cases'] <0 , 0, inplace=True)  #remove negative values. There seems to be a bug in the original data which needs to be fixed.

res.to_csv(path + 'covid_2020_2023.csv')